# Welcome to __PERFUMEme.py__ 🧪👃🧴⚛️⚗️

## Introduction
Perfumes are a part of our everyday lives—whether it’s a subtle floral hint or a bold, spicy statement, most people have a signature scent that others come to recognize them by. But have you ever wondered what’s actually in your favorite fragrance? Perfumes are complex mixtures of countless molecules, and each one plays a unique role in shaping the scent you love.

So how can you figure out which molecule is responsible for that special note in your perfume? And once you have a molecule in mind, how do you know if it’s safe, aromatic, or even used in other fragrances?

Meet __PERFUMEme.py__, — a Python package that helps demystify the chemistry behind your scent. Simply input a molecule, and the tool will tell you:

- Whether the molecule is fragrant or toxic
- Its key physical properties, such as boiling point
- How it evaporates over time, via an evaporation model
- And best of all, which perfumes (when data allows) contain that exact molecule

Unlock the secrets behind the scents — one molecule at a time.

The main chemistry involved is...

## Material and methods

### Data Sources 🗂
Molecular data were retrieved from PubChem, a publicly accessible chemical database maintained by the National Center for Biotechnology Information (NCBI). The PubChem PUG REST API was used to access compound properties, such as their molecular names, SMILES strings, boiling point, enthalpy of vaporisation and pressure value.

The fragrance data, including perfume names, brands, notes, and associated molecules, were compiled manually and stored in a local JSON file. This file served as the data set for the perfume_molecule.py functions.

### Package Structure 𝌤
The __PERFUMEme__ package was implemented in Python and organized into several functional modules:

- scraper.py: includes functions used to add the molecules listed in the perfumes in perfume.json to a new database and on to addtheir smiles from PubChem, used withodors.csv to add their respective odors (if they had smell).
- utils.py: contains utility functions for validating molecule data, handling API errors, and processing JSON inputs.
- data/: directory containing the JSON files with structured perfume and molecular data.

The package was developed to be modular, allowing users to integrate new data sources or extend the current capabilities without modifying core functions. For example, adding a molecule with its odor and SMILES to molecules.json.

## Results and discussion

### Does it smell?

The first step was

- wanted database with all molecules -> way too many, would be too heavy and not possible to do a search of all molecules in the world. Instead for some functions, go directly on the web. If don't know SMILES, database with most common molecules in perfumes
- database with most common molecules and their smiles
- database with 29 perfumes and which molecules they contain
- database with odor of these most common molecules -> maybe just put it in the smiles database?

Important functions:
- has a smell
- is toxic
- evaporation trace
- usable in perfume
- match mol to perfume
- match mol to odor
- combination

In [10]:
import json
from src.perfumeme.perfume_molecule import match_molecule_to_perfumes

with open("/Users/lilia/Documents/EPFL/BA.4/prog/project/PERFUMEme/data/perfumes.json") as f:
    perfumes = json.load(f)

# Find all perfumes containing Linalool
linalool_perfumes = match_molecule_to_perfumes("Linalool")
print(linalool_perfumes)


ModuleNotFoundError: No module named 'src'

In [9]:
from src.perfumeme.main_functions import has_a_smell, is_toxic_skin, evaporation_trace

evaporation_trace("water")

ModuleNotFoundError: No module named 'src'